In [11]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import io

In [33]:
log_file = 'webruns/round2_scrape.log'

In [38]:
import json
import pandas as pd
import io

with open(log_file, 'r') as file:
    file_content = file.read()
    sections = file_content.split('Sandbox logs:')[1].split('Activities log:')
    
    # Parse the Sandbox logs section into a JSON array
    sandbox_logs = []
    logs_data = sections[0].strip()
    
    # Split the logs data into individual JSON objects
    start_index = 0
    while start_index < len(logs_data):
        if logs_data[start_index] == '{':
            end_index = logs_data.find('}', start_index) + 1
            log_entry = logs_data[start_index:end_index]
            sandbox_logs.append(json.loads(log_entry))
            start_index = end_index
        else:
            start_index += 1
    
    # Create a DataFrame from the parsed JSON data
    df_sandbox = pd.DataFrame(sandbox_logs)
    
    # Extract the ORCHIDS POSITION value from the lambdaLog column
    df_sandbox['orchids_position'] = df_sandbox['lambdaLog'].str.extract(r'ORCHIDS POSITION: (\-?\d+)').astype(int)
    
    # Select the desired columns for the final DataFrame
    df_sandbox = df_sandbox[['timestamp', 'orchids_position']]
    
    activities_log = sections[1].split('Trade History:')[0]
    df_activities = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    
    trade_json = pd.json_normalize(json.loads(sections[1].split('Trade History:')[1])).to_json()


In [39]:
df_sandbox

,timestamp,orchids_position
0,0,0
1,100,0
2,200,-70
3,300,-30
4,400,0
...,...,...
995,99500,-77
996,99600,0
997,99700,-70
998,99800,-30


In [43]:
import json
import pandas as pd
import io

with open(log_file, 'r') as file:
    file_content = file.read()
    sections = file_content.split('Sandbox logs:')[1].split('Activities log:')
    
    # Parse the Sandbox logs section into a JSON array
    sandbox_logs = []
    logs_data = sections[0].strip()
    
    # Split the logs data into individual JSON objects
    start_index = 0
    while start_index < len(logs_data):
        if logs_data[start_index] == '{':
            end_index = logs_data.find('}', start_index) + 1
            log_entry = logs_data[start_index:end_index]
            sandbox_logs.append(json.loads(log_entry))
            start_index = end_index
        else:
            start_index += 1
    
    # Create a DataFrame from the parsed JSON data
    df_sandbox = pd.DataFrame(sandbox_logs)
    
    # Extract the ORCHIDS POSITION value from the lambdaLog column
    df_sandbox['orchids_position'] = df_sandbox['lambdaLog'].str.extract(r'ORCHIDS POSITION: (\-?\d+)').astype(int)
    
    # Extract the IMPLIED_BID value from the lambdaLog column
    df_sandbox['implied_bid'] = df_sandbox['lambdaLog'].str.extract(r'IMPLIED_BID: (\d+\.\d+)').astype(float)
    
    # Extract the IMPLIED_ASK value from the lambdaLog column
    df_sandbox['implied_ask'] = df_sandbox['lambdaLog'].str.extract(r'IMPLIED_ASK: (\d+\.\d+)').astype(float)
    
    # Extract the CURR EDGE value from the lambdaLog column
    df_sandbox['curr_edge'] = df_sandbox['lambdaLog'].str.extract(r'CURR EDGE: (\d+)').astype(int)
    
    # Select the desired columns for the final DataFrame
    df_sandbox = df_sandbox[['timestamp', 'orchids_position', 'implied_bid', 'implied_ask', 'curr_edge']]
    
    activities_log = sections[1].split('Trade History:')[0]
    df_activities = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    
    trade_json = pd.json_normalize(json.loads(sections[1].split('Trade History:')[1])).to_json()

# Display the DataFrame
display(df_sandbox)

,timestamp,orchids_position,implied_bid,implied_ask,curr_edge
0,0,0,1086.0,1094.9,2
1,100,0,1087.0,1094.9,2
2,200,-70,1085.5,1093.4,2
3,300,-30,1085.5,1093.9,2
4,400,0,1086.0,1093.9,2
...,...,...,...,...,...
995,99500,-77,1060.3,1067.1,2
996,99600,0,1061.3,1068.1,2
997,99700,-70,1061.3,1068.6,2
998,99800,-30,1061.3,1068.1,2


In [44]:
df_sandbox.isna().mean()

timestamp           0.0
orchids_position    0.0
implied_bid         0.0
implied_ask         0.0
curr_edge           0.0
dtype: float64

In [56]:
def parse_log_file(log_file, product):
    with open(log_file, "r") as file:
        file_content = file.read()
    sections = file_content.split("Sandbox logs:")[1].split("Activities log:")
    activities_log = sections[1].split("Trade History:")[0]
    df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_json = pd.json_normalize(
        json.loads(sections[1].split("Trade History:")[1])
    ).to_json()
    sandbox_logs = []
    logs_data = sections[0].strip()
    start_index = 0
    while start_index < len(logs_data):
        if logs_data[start_index] == "{":
            end_index = logs_data.find("}", start_index) + 1
            log_entry = logs_data[start_index:end_index]
            sandbox_logs.append(json.loads(log_entry))
            start_index = end_index
        else:
            start_index += 1
    df_sandbox = pd.DataFrame(sandbox_logs)
    if "lambdaLog" in df_sandbox.columns:
        if df_sandbox["lambdaLog"].str.contains("IMPLIED_BID").any():
            implied_bid = pd.to_numeric(df_sandbox["lambdaLog"].str.extract(r"IMPLIED_BID: (\d+\.\d+)")[0], errors='coerce')
            implied_bid_df = pd.DataFrame({"timestamp": df_sandbox["timestamp"], "implied_bid": implied_bid, "product": "ORCHIDS"})
            df = pd.merge(df, implied_bid_df, on=["timestamp", "product"], how="left")
        if df_sandbox["lambdaLog"].str.contains("IMPLIED_ASK").any():
            implied_ask = pd.to_numeric(df_sandbox["lambdaLog"].str.extract(r"IMPLIED_ASK: (\d+\.\d+)")[0], errors='coerce')
            implied_ask_df = pd.DataFrame({"timestamp": df_sandbox["timestamp"], "implied_ask": implied_ask, "product": "ORCHIDS"})
            df = pd.merge(df, implied_ask_df, on=["timestamp", "product"], how="left")
    return df, trade_json, df_sandbox.to_json()


In [57]:
df = parse_log_file(log_file, "ORCHIDS")[0]

In [58]:
df

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,implied_bid,implied_ask
0,1,0,AMETHYSTS,9998,3,9996.0,1.0,9995.0,21.0,10004,1,10005.0,21.0,NaN,NaN,10001.0,0.000000,NaN,NaN
1,1,0,ORCHIDS,1094,10,1093.0,3.0,1086.0,41.0,1102,10,1104.0,3.0,1111.0,41.0,1098.0,0.000000,1086.0,1094.9
2,1,0,STARFRUIT,5048,1,5047.0,22.0,NaN,NaN,5054,22,NaN,NaN,NaN,NaN,5051.0,0.000000,NaN,NaN
3,1,100,STARFRUIT,5047,26,NaN,NaN,NaN,NaN,5052,5,5054.0,26.0,NaN,NaN,5049.5,0.000000,NaN,NaN
4,1,100,ORCHIDS,1094,6,1093.0,2.0,1087.0,51.0,1102,6,1103.0,2.0,1110.0,51.0,1098.0,0.000000,1087.0,1094.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,99800,AMETHYSTS,9996,1,9995.0,25.0,NaN,NaN,10004,1,10005.0,25.0,NaN,NaN,10000.0,1599.000000,NaN,NaN
2996,1,99800,ORCHIDS,1068,8,1067.0,3.0,1061.0,49.0,1075,8,1076.0,3.0,1083.0,49.0,1071.5,50475.585938,1061.3,1068.1
2997,1,99900,STARFRUIT,5043,28,NaN,NaN,NaN,NaN,5050,28,NaN,NaN,NaN,NaN,5046.5,1267.367188,NaN,NaN
2998,1,99900,ORCHIDS,1067,7,1066.0,4.0,1059.0,49.0,1074,7,1075.0,4.0,1081.0,49.0,1070.5,50771.585938,1059.8,1066.6


In [59]:
df_orchids = df[df['product'] == 'ORCHIDS']

In [60]:
df_orchids

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,implied_bid,implied_ask
1,1,0,ORCHIDS,1094,10,1093.0,3.0,1086.0,41.0,1102,10,1104.0,3.0,1111.0,41.0,1098.0,0.000000,1086.0,1094.9
4,1,100,ORCHIDS,1094,6,1093.0,2.0,1087.0,51.0,1102,6,1103.0,2.0,1110.0,51.0,1098.0,0.000000,1087.0,1094.9
7,1,200,ORCHIDS,1093,6,1092.0,3.0,1085.0,49.0,1101,6,1102.0,3.0,1108.0,49.0,1097.0,252.000000,1085.5,1093.4
11,1,300,ORCHIDS,1093,10,1092.0,4.0,1085.0,52.0,1101,10,1102.0,4.0,1109.0,52.0,1097.0,285.000000,1085.5,1093.9
13,1,400,ORCHIDS,1093,10,1092.0,3.0,1086.0,51.0,1101,10,1102.0,3.0,1109.0,51.0,1097.0,285.000000,1086.0,1093.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986,1,99500,ORCHIDS,1067,5,1066.0,4.0,1060.0,43.0,1074,5,1075.0,4.0,1082.0,43.0,1070.5,50290.585938,1060.3,1067.1
2988,1,99600,ORCHIDS,1068,7,1067.0,2.0,1061.0,46.0,1075,7,1076.0,2.0,1083.0,46.0,1071.5,50290.585938,1061.3,1068.1
2991,1,99700,ORCHIDS,1068,5,1067.0,3.0,1061.0,48.0,1076,5,1077.0,3.0,1083.0,48.0,1072.0,50388.585938,1061.3,1068.6
2996,1,99800,ORCHIDS,1068,8,1067.0,3.0,1061.0,49.0,1075,8,1076.0,3.0,1083.0,49.0,1071.5,50475.585938,1061.3,1068.1


1       13.0
4        8.0
7        9.0
11      14.0
13      13.0
        ... 
2986     9.0
2988     9.0
2991     8.0
2996    11.0
2998    11.0
Length: 1000, dtype: float64